In [4]:
from bs4 import BeautifulSoup
import requests
import time
import random
f = open("/Users/digitalfirst/Desktop/HW3 ADM/movies3.html")

soup = BeautifulSoup(f)
listUrl_Movies3=[]
for link in soup.select('a'):
    listUrl_Movies3.append(link.text)

In [ ]:
def downloadFile(index):

    for index in range(len(listUrl_Movies3)):
        
        t2=1200
        try:
            t1 = random.randint(1,5)
            time.sleep(t1)
            url=listUrl_Movies3[index]
            response = requests.get(url)
            name="aritcle_"
            extension=".html"
            file="{}{}{}".format(name,index,extension)
            with open(file,'wb') as f: 
                f.write(response.content)  

        except response.status_code as e:
            print("exception")
            if e==492:
                time.sleep(t2)
                downloadFile(index+1)
            elif e==200:
                soup = BeautifulSoup(listUrl_Movies3[1])
                
                with open(file,'w') as f: 
                    f.write(soup.text)
                downloadFile(index+1)
            else:
                continue



In [ ]:
downloadFile(0)

In [ ]:
import csv
import pandas as pd
import os.path
#define column of our dataframe
df=pd.DataFrame(columns=['title', 'intro', 'plot','film_name','producer','director','writer','starring','music','release date','runtime','country','language','budget'])


for index in range(len(listUrl_Movies3)):
    
    title=''
    plot=''
    intro=''
    title_name='NA'
    producer='NA'
    director='NA'
    writer='NA'
    starring='NA'
    music='NA'
    release_date='NA'
    runtime='NA'
    country='NA'
    language='NA'
    budget='NA'
    
    
    #define name of the file that we want to find (in my case: in the same directory)
    name="aritcle_"
    extension=".html"
    file="{}{}{}".format(name,index,extension)
    
    #check if this file exists
    if not os.path.isfile(file):
        continue
        
    #open file   
    response2 = open(file)
    soup = BeautifulSoup(response2)
    #take title.
    title=soup.title.text.rsplit(' ', 2)[0]
    
    #take all p in intro(firt section)
    if soup.find('span', attrs={'class': 'mw-headline'}):
        
        heading = soup.find('span', attrs={'class': 'mw-headline'})
        paragraphs = heading.find_all_previous('p')
        for p in paragraphs: 
            intro = p.text + intro
            
     
        #take all p in 'plot'(second section)
        b=True
        if soup.find('span', attrs={'class': 'mw-headline'}): 
            
            heading = soup.find('span', attrs={'class': 'mw-headline'})
            if heading.find_all_next('p'):
                
                paragraphs = heading.find_all_next('p')
                for p in paragraphs: 
                    # print (team.text)
                    plot=plot+p.text
                    if p.next_sibiling:
                    
                        if not p.next_sibling.name=='p':
                            b=False
                        if not b:
                            continue
                    else:
                        continue
            else:
                plot="NAN"
        else:
            plot="NAN"
    
    
    
    
    else:
        intro="NAN"
        plot="NAN"
    
    #here: code to get info about infobox from every page    
    if soup.find('table', attrs={'class': 'infobox vevent'}):
        
        table = soup.find('table', attrs={'class': 'infobox vevent'})  
    
        if table.find('th', attrs={'class': 'summary'}):
        
            x=table.find('th', attrs={'class': 'summary'})
            title_name=x.text.strip()
        
        for cell in table.find_all('th'):
        
            if cell.find_next_sibling('td'):
                a=cell.find_next_sibling('td')
                if cell.text.strip()=='Directed by':
                    director=a.text.strip()
                elif cell.text.strip()=='Produced by':
                
                    producer=a.text.strip()
                elif cell.text.strip()=='Written by':
                
                    writer=a.text.strip()
                elif cell.text.strip()=='Starring':
              
                    starring=a.text.strip()
                elif cell.text.strip()=='Music by':
                
                    music=a.text.strip()
                elif cell.text.strip()=='Release date':
                    release_date=a.text.strip()   
                elif cell.text.strip()=='Running time':
                
                    runtime=a.text.strip()
                elif cell.text.strip()=='Country':
              
                    country=a.text.strip()
                elif cell.text.strip()=='Language':
              
                    language=a.text.strip()
                elif cell.text.strip()=='Budget':
              
                    budget=a.text.strip()
            else:
                continue
            
    
    
    
    #put all infos in movie list
    movie=[title,intro,plot,title_name,producer,director,writer,starring,music,release_date,runtime,country,language,budget]
    #update dataframe with this list
    extension2=".tsv"
    file="{}{}{}".format(name,index,extension2)
   
    movieTitle=["title","intro","plot","title_name","producer","director","writer","starring","music","release_date","runtime","country","language","budget"]
    with open('tsv/'+file, 'w', newline='') as f_output:
        tsv_output = csv.writer(f_output, delimiter='\t')
        tsv_output.writerow(movieTitle)
        tsv_output.writerow(movie)
    df.loc[index] = movie

In [ ]:
import string

from shutil import move
from nltk.tokenize import RegexpTokenizer
#create tsv file in 'tsv_correct' directory wehere we have preprocessed the tsv file (just created in parser.py)
tokenizer = RegexpTokenizer(r'\w+')
name="aritcle_"
extension2=".tsv"
exclude = string.punctuation
for index in range(len(listUrl_Movies3)):



    file="{}{}{}".format(name,index,extension2)
    with open("tsv/"+file,"r") as tsvfile, open("tsv_correct/"+file,"w") as outfile:
        tsvreader = csv.reader(tsvfile, delimiter="\t")
        tsvwriter = csv.writer(outfile, delimiter="\t")
        for row in tsvreader:
            for i in range(len(row)):
                #take every words, deleting ountuaction and other symbols
                row[i] = tokenizer.tokenize(row[i])
                #remove duplicate case-insensitive elements
                row[i]= list(set(map(str.lower, row[i])))
                #row[i] = row[i].translate({ord(c): None for c in string.punctuation})
                
            tsvwriter.writerow(row)
            

In [ ]:
import ast
from itertools import islice
import csv
#create vocabulary and save it on vocabulary.tsv
            

dict1 = dict()
term_id=0
present=False
with open('tsv/vocobulary.tsv', 'w', newline='') as f_output:
        tsv_vocabulary = csv.writer(f_output, delimiter='\t')
        tsv_vocabulary.writerow(['word','term_id'])
        name="aritcle_"
        extension2=".tsv"
        h=0
        for index in range(len(listUrl_Movies3)):
            h+=1
            print(h)
            file="{}{}{}".format(name,index,extension2)
            with open("tsv_correct/"+file,"r") as tsvfile:
                data_list = list(csv.reader(tsvfile, delimiter="\t"))
                tsvreader = csv.reader(tsvfile, delimiter="\t")
                #put in intro a list of all words that we have in intro of i-th page
                intro=data_list[1][2]
                intro = ast.literal_eval(intro)
                #put in plot a list of all words that we have in plot of i-th page
                plot=data_list[1][1]
                plot = ast.literal_eval(plot)
                
                #put in text, a list that contains all words that are in plot and word for every page (no duplicate)
                text=plot+intro
                text= list(set(map(str.lower, text)))
                
                #put in dict1 every words with its term_id (no duplicate)
                for i in text:
                    if i in dict1:    
                        continue
                    else:
                        dict1[i]=term_id
                        term_id+=1
                
        #put dict1 element in vocabulary.tsv file                
        for key, val in dict1.items():
                    tsv_vocabulary.writerow([key, val])




In [ ]:
import ast
from itertools import islice
import csv

#create index and save it on index.tsv                

dict2 = {}
count=0
present=False
with open('tsv/vocobulary.tsv', 'r', newline='') as f_output:
        tsv_vocabulary = list(csv.reader(f_output, delimiter='\t'))
        name="aritcle_"
        extension2=".tsv"
        h=0
        for row in tsv_vocabulary:
            
            dict2[row[1]]=[]
        for index in range(len(listUrl_Movies3)):
            h+=1
            print(h)
            file="{}{}{}".format(name,index,extension2)
            with open("tsv_correct/"+file,"r") as tsvfile:
                data_list = list(csv.reader(tsvfile, delimiter="\t"))
                tsvreader = csv.reader(tsvfile, delimiter="\t")
                intro=data_list[1][1]
                intro = ast.literal_eval(intro)
                plot=data_list[1][2]
                plot = ast.literal_eval(plot)
                text=plot+intro
                text= list(set(map(str.lower, text)))
                
                #for evry words in plot adn intro (for every page) we get every word. From every word we get its term_id and put it whit their occurences (document_id) in dict2
                for i in text:
                    for row in tsv_vocabulary:
                        if i==row[0]:
                            doc="document_"
                            name2="{}{}".format(doc,index)
                            
                            dict2[row[1]].append(name2)
                            break
                        else:
                            continue
                            
        #put dict2 in index.tsv file. In. evry row we have a single term_id with occurences of respective word.
        with open('tsv/index2.tsv', 'w', newline='') as f_output:
            tsv_vocabulary = csv.writer(f_output, delimiter='\t')           
            for key, val in dict2.items():
                tsv_vocabulary.writerow([key, val])           
                    




In [7]:
import csv
import sys
import ast
#define function that allows us to calculate a list that is an intersection from two list
def intersection(lst1, lst2): 
    return list(set(lst1) & set(lst2)) 

#INPUT QUERY
words='enormous damage unless something is done immediately'
#we use dict3 to store term_id and its respective documents_id
dict3={}
##we use dict4 to store evry word and its respective documents_id
dict4={}
csv.field_size_limit(sys.maxsize)
#in listWords we have a list that contains all words about inout query
listWords = words.split()
listWords=[x.lower() for x in listWords]

#with vocabulary.tsv we start to build a dict3 with term_id for every words in wordsList
with open('tsv/vocobulary.tsv', 'r', newline='') as f_output:
    tsv_vocabulary = list(csv.reader(f_output, delimiter='\t'))
    for word in listWords:
        word=word.lower()
        present=False
        for row in tsv_vocabulary:
            if word.lower()==row[0]:
                dict3[row[1]]=[]
                present=True
        #case where word is not in vocabulary
        if present==False:
            dict4[word]=[]
            
    #we continue to match dicumnets_id to every term_id in dict3
    with open('tsv/index2.tsv', 'r', newline='') as f_output:
        tsv_index = list(csv.reader(f_output, delimiter='\t'))
        for k in dict3.keys():  
            for row in tsv_index:
                if row[0]==k:
                    dict3[k]=row[1]
                    continue
    
    
    #finally we build dict4 where evry word matches to respective documents_id
    for k in dict3.keys():
        
        for row in tsv_vocabulary:
            if k==row[1]:
                dict4[row[0]]=dict3[row[1]]
            
    document=ast.literal_eval(dict4[listWords[0]])         
    #interection between every list in values dict4. In this way we have documnets_id where all words (in query input) are present
    for value in dict4.values():
        document=intersection(document,ast.literal_eval(value))
    print(document)
          
                
    

['document_12']


In [8]:
import pandas as pd

#build the dataframe with info for every documents_id
df=pd.DataFrame(columns=['title', 'intro', 'url'])
for index in range(len(document)):
    #get id of documnets_is
    numberDocument=document[index][9:]
    #get wikipedia url
    url=listUrl_Movies3[int(numberDocument)]
    name="aritcle_"
    extension2=".tsv"
    index=int(numberDocument)
    file="{}{}{}".format(name,index,extension2)
    #get info about title and intro for evert film that corresponds to every documents_id
    with open('tsv/'+file, 'r', newline='') as f_output:
        tsv_index = list(csv.reader(f_output, delimiter='\t'))
        title=tsv_index[1][3]
        intro=tsv_index[1][1]
        film=[title,intro,url]
        #put all info for every film in a single row of df dataframe
        df.loc[index] = film

In [9]:
df

,title,intro,url
12,"Digby, the Biggest Dog in the World","\nDigby, the Biggest Dog in the World is the t...","https://en.wikipedia.org/wiki/Digby,_the_Bigge..."
